# 네이버 영화리뷰 감정분석 with XGBoost
TF-IDF 기법 참고(링크) : https://wiserloner.tistory.com/944

# 준비
라이브러리, 파라미터 세팅

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 로드

In [3]:
# 학습 데이터 로드
train_df = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
# 테스트(캐글) 데이터 로드
test_df = pd.read_csv(DATA_IN_PATH + 'ko_data.csv', encoding = 'cp949')
test_df.columns = ['id','document']  # 전처리 일괄 수행을 위해 컬럼명 변경(학습 데이터셋과 동일하게)
test_df.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


# 데이터 전처리

In [5]:
def data_cleansing(train_data, test_data, con=0):  # 0: Null치환(공백), 1: 한글이외제거 & Null(공백)제거, 2: 숫자제거 & Null치환(공백)
    global train_df
    global test_df
    
    train_data = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', sep='\t')
    test_data = pd.read_csv(DATA_IN_PATH + 'ko_data.csv', encoding = 'cp949')
    test_data.columns = ['id','document']  # 전처리 일괄 수행을 위해 컬럼명 변경(학습 데이터셋과 동일하게)
    train_df = train_data.copy()
    test_df = test_data.copy()
    
    if con == 1:  # 한글 이외 제거 & Null(공백) 제거, row 제거 有
        train_df.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복 내용 제거
        train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행(한글 이외 제거)
        train_df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
        train_df = train_df.dropna(how='any') # Null 값 제거(리뷰 Null은 긍/부정 의미 없음)
        
        test_df.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복 내용 제거
        test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행(한글 이외 제거)
        test_df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
        test_df = test_df.dropna(how='any') # Null 값 제거(리뷰 Null은 긍/부정 의미 없음)
        
    elif con == 2:  # 숫자 제거 & Null 치환(공백), row 제거 없음
        # 리뷰 텍스트를 가지는 'document' 컬럼에 Null이 일부 존재하므로 이 값은 공백으로 변환
        # 정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미)
        train_df = train_df.fillna(' ')
        train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )
        
        test_df = test_df.fillna(' ')
        test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )
        
    else:
        train_df = train_df.fillna(' ')
        test_df = test_df.fillna(' ')
    
    print('전처리 후 학습 데이터 개수 :',len(train_df))
    print('전처리 후 테스트 데이터 개수 :',len(test_df))


In [6]:
data_cleansing(train_df, test_df, con=0)  # 0: Null치환(공백), 1: 한글이외제거 & Null(공백)제거, 2: 숫자제거 & Null치환(공백)

전처리 후 학습 데이터 개수 : 150000
전처리 후 테스트 데이터 개수 : 11187


In [7]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [8]:
test_df.head()

,id,document
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.


In [9]:
# 학습 데이터 레이블 넘파이 배열 처리
y_train = np.array(train_df['label'])

# TF-IDF를 활용한 벡터화

TF-IDF를 통해 단어를 벡터화할 것이고, 먼저 각 문장을 한글 형태소 분석을 통해 형태소 단어로 토큰화한다. 한글 형태소 엔진은 SNS 분석에 적합한 Twitter 클래스를 이용한다. Twitter 객체의 morphs() 메소드를 이용하면 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체로 반환한다. 문장을 형태소 단어 형태로 반환하는 별도의 tokenizer 함수를 tw_tokenizer()라는 이름으로 생성한다. 이 함수는 사이킷런의 TfidfVectorizer 클래스의 tokenizer로 사용된다.

In [10]:
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온  텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [11]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer

start = time.time()  # 시작 시간 저장

# TfidfVectorizer를 이용해 학습 데이터를 TF-IDF 값으로 피처 변환
# Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용. ngram_range는 (1, 2)
vectorizer = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
vectorizer.fit(train_df['document'])
train_data_features = vectorizer.transform(train_df['document'])

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환
test_data_features = vectorizer.transform(test_df['document'])

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
print()
print(train_data_features.shape)
print(test_data_features.shape)
print(type(train_data_features))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


time : 468.5085115432739

(150000, 130333)
(11187, 130333)
<class 'scipy.sparse.csr.csr_matrix'>


# (Baseline) 학습과 검증 데이터 분리 후 모델 학습

In [12]:
from sklearn.model_selection import train_test_split

train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y_train, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier

start = time.time()  # 시작 시간 저장

# XGBoost 사용해서 감성 분석 분류 수행
clf = XGBClassifier(random_state=RANDOM_SEED)
clf.fit(train_input, train_label)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 

time : 5.674331188201904


In [14]:
print("Accuracy: %f" % clf.score(eval_input, eval_label))  # 검증 함수로 정확도 측정

Accuracy: 0.789800


# 모델(GridSearch) 구현 및 학습 

In [15]:
'''
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 사용해서 감성 분석 분류 수행
xgb_clf = XGBClassifier(random_state=RANDOM_SEED)

# 파라미터 최적화를 위해 GridSearchCV를 이용
params = { 'min_child_weight' : [1],  # 과적합 심할 경우 값 높여야 함
           'max_depth' : [4,6,8],       # 과적합 심할 경우 값 낮춰야 함
           'gamma' : [0] }            # 과적합 심할 경우 값 높여야 함

grid_cv = GridSearchCV(xgb_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(train_data_features, y)
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))
'''

"\nimport xgboost as xgb\nfrom xgboost import XGBClassifier\nfrom sklearn.model_selection import GridSearchCV\n\n# 랜덤 포레스트 사용해서 감성 분석 분류 수행\nxgb_clf = XGBClassifier(random_state=RANDOM_SEED)\n\n# 파라미터 최적화를 위해 GridSearchCV를 이용\nparams = { 'min_child_weight' : [1],  # 과적합 심할 경우 값 높여야 함\n           'max_depth' : [4,6,8],       # 과적합 심할 경우 값 낮춰야 함\n           'gamma' : [0] }            # 과적합 심할 경우 값 높여야 함\n\ngrid_cv = GridSearchCV(xgb_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)\ngrid_cv.fit(train_data_features, y)\nprint(grid_cv.best_params_, round(grid_cv.best_score_, 4))\n"

# Original 테스트 데이터셋으로 성능 평가

In [16]:
# 검증 데이터 로드
dev_df = pd.read_csv(DATA_IN_PATH + 'ratings_test.txt', sep='\t')
dev_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [17]:
# 리뷰 텍스트를 가지는 'document' 컬럼에 Null이 일부 존재하므로 이 값은 공백으로 변환
dev_df = dev_df.fillna(' ')
# # 정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미)
# dev_df['document'] = dev_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

dev_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [18]:
# 학습 데이터를 적용한 TfidfVectorizer를 이용해 검증 데이터를 TF-IDF 값으로 피처 변환
dev_data_features = vectorizer.transform(dev_df['document'])

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score

# # classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
# best_estimator = grid_cv.best_estimator_
# preds = best_estimator.predict(dev_data_features)

preds = clf.predict(dev_data_features)

In [20]:
confusion = confusion_matrix(dev_df['label'], preds)

print('오차 행렬')
print(confusion,'\n')

print('XGBoost 정확도  :', accuracy_score(dev_df['label'], preds))
print('XGBoost 정밀도  :', precision_score(dev_df['label'], preds))
print('XGBoost 재현율  :', recall_score(dev_df['label'], preds))
print('XGBoost F1스코어:', f1_score(dev_df['label'], preds))
print('XGBoost AUC     :', roc_auc_score(dev_df['label'], preds))

오차 행렬
[[20774  4053]
 [ 6635 18538]] 

XGBoost 정확도  : 0.78624
XGBoost 정밀도  : 0.8205922712584658
XGBoost 재현율  : 0.7364239462916617
XGBoost F1스코어: 0.7762331463026547
XGBoost AUC     : 0.786587129225905


# (참고) 제출 파일 생성

In [21]:
# 테스트 데이터의 리뷰 부분을 리스트 처리
test_reviews = list(test_df['id'])

test_preds = clf.predict(test_data_features)

# 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
output = pd.DataFrame( data={"Id": test_reviews, "Predicted": test_preds} )
output.head()

,Id,Predicted
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [23]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv 파일 생성
output.to_csv(DATA_OUT_PATH + "NSMC_XGBoost.csv", index = False)

### 캐글 제출 결과
**[2020.12.20]**<br>
0.76238

# (참고) 리뷰 예측하기

In [24]:
# 입력을 리스트 형태로 넣어야 함
# 참고 : https://m.blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220985170721&proxyReferer=https:%2F%2Fwww.google.com%2F
def sentiment_predict(new_sentence):
    label = {0: '부정', 1: '긍정'}
    new_sentence = vectorizer.transform(new_sentence)  # TF-IDF 값으로 피처 변환
    print('예측: %s\n확률: %.3f%%' %(label[clf.predict(new_sentence)[0]], np.max(clf.predict_proba(new_sentence))*100))

In [25]:
sentiment_predict(['이 영화 개꿀잼 ㅋㅋㅋ'])

예측: 부정
확률: 52.540%


In [26]:
sentiment_predict(['이 영화 핵노잼 ㅠㅠ'])

예측: 부정
확률: 72.889%


In [27]:
sentiment_predict(['견디기 힘드네요'])

예측: 부정
확률: 52.540%


---